In [1]:
# step 1:读取交割单
import pymongo
import pandas as pd
strategy_id = 2
pd.set_option("display.width", 160)
client = pymongo.MongoClient("mongodb://192.168.0.101,192.168.0.102")
c = client.get_database("signal").get_collection("trade")
trades = c.find({"strategy": strategy_id}, {"_id": 0})
tdf = pd.DataFrame(list(trades))
if len(tdf):
    tdf["datetime"] = tdf["trading_datetime"]

In [13]:
tdf.head()

,commission,datetime,exec_id,last_price,last_quantity,order_book_id,order_id,position_effect,side,strategy,symbol,tax,trading_datetime,transaction_cost
0,79.7880,2016-10-26 15:00:00,1510296142,9.15,10900,000001.XSHE,1510296141,None,BUY,2,平安银行,0.000,2016-10-26 15:00:00,79.7880
1,81.1832,2017-03-17 15:00:00,1510296143,9.31,10900,000001.XSHE,1510296143,None,SELL,2,平安银行,101.479,2017-03-17 15:00:00,182.6622
2,80.8080,2017-05-26 15:00:00,1510296144,9.10,11100,000001.XSHE,1510296144,None,BUY,2,平安银行,0.000,2017-05-26 15:00:00,80.8080
3,79.7880,2016-10-26 15:00:00,1510296176,9.15,10900,000001.XSHE,1510296175,None,BUY,2,平安银行,0.000,2016-10-26 15:00:00,79.7880
4,81.1832,2017-03-17 15:00:00,1510296177,9.31,10900,000001.XSHE,1510296177,None,SELL,2,平安银行,101.479,2017-03-17 15:00:00,182.6622


In [4]:
from fxdayu_betaman.calculator.analysis import Performance
perf = Performance(tdf,10000000) # 传入交割单 传入起始资金

In [16]:
perf.transactions

,symbol,amount,price
datetime,,,
2016-10-26 15:00:00,000001.XSHE,10900,9.15
2016-10-26 15:00:00,000001.XSHE,10900,9.15
2016-10-26 15:00:00,000001.XSHE,10900,9.15
2016-10-26 15:00:00,000001.XSHE,10900,9.15
2016-10-26 15:00:00,000001.XSHE,10900,9.15
2017-03-17 15:00:00,000001.XSHE,-10900,9.31
2017-03-17 15:00:00,000001.XSHE,-10900,9.31
2017-03-17 15:00:00,000001.XSHE,-10900,9.31
2017-03-17 15:00:00,000001.XSHE,-10900,9.31


In [8]:
# 仓位(按品种)
perf.position.head()

datetime  cumsum_quantity
order_book_id order_id                                     
000001.XSHE   0.0      2016-10-26 15:00:00          10900.0
              1.0      2016-10-26 15:00:00          21800.0
              2.0      2016-10-26 15:00:00          32700.0
              3.0      2016-10-26 15:00:00          43600.0
              4.0      2016-10-26 15:00:00          54500.0

In [9]:
# 证券市值（按品种）
perf.market_value.head()

datetime  market_value
order_book_id order_id                                  
000001.XSHE   0.0      2016-10-26 15:00:00       99735.0
              1.0      2016-10-26 15:00:00      199470.0
              2.0      2016-10-26 15:00:00      299205.0
              3.0      2016-10-26 15:00:00      398940.0
              4.0      2016-10-26 15:00:00      498675.0

In [10]:
# 账面盈亏-含手续费损失（按品种）
perf.pnl.head()

datetime      pnl
order_book_id order_id                             
000001.XSHE   0.0      2016-10-26 15:00:00  -79.788
              1.0      2016-10-26 15:00:00 -159.576
              2.0      2016-10-26 15:00:00 -239.364
              3.0      2016-10-26 15:00:00 -319.152
              4.0      2016-10-26 15:00:00 -398.940

In [11]:
# 账面收益-不含手续费（按品种）
perf.profits.head()

datetime  profits
order_book_id order_id                             
000001.XSHE   0.0      2016-10-26 15:00:00      NaN
              1.0      2016-10-26 15:00:00      NaN
              2.0      2016-10-26 15:00:00      NaN
              3.0      2016-10-26 15:00:00      NaN
              4.0      2016-10-26 15:00:00      NaN

In [18]:
# 持仓均价（按品种）
perf.average_price.head()

datetime  avg_price
order_book_id order_id                               
000001.XSHE   0.0      2016-10-26 15:00:00       9.15
              1.0      2016-10-26 15:00:00       9.15
              2.0      2016-10-26 15:00:00       9.15
              3.0      2016-10-26 15:00:00       9.15
              4.0      2016-10-26 15:00:00       9.15

In [19]:
# 每日证券市值dataframe（所有品种+可用现金 作为column）
perf.daily_mv_df.head()

,000001.XSHE,cash
datetime,,
2016-10-26,498675.0,9500926.06
2016-10-27,499220.0,9500926.06
2016-10-28,499765.0,9500926.06
2016-10-31,498675.0,9500926.06
2016-11-01,498130.0,9500926.06


In [20]:
# 每日证券市值总值
perf.daily_security_value.head()

datetime
2016-10-26    498675.0
2016-10-27    499220.0
2016-10-28    499765.0
2016-10-31    498675.0
2016-11-01    498130.0
Name: daily_security_value, dtype: float64

In [21]:
# 每日持仓（时间+品种的multiindex）
perf.daily_position.head()

datetime    order_book_id
2016-10-26  000001.XSHE      54500.0
2016-10-27  000001.XSHE      54500.0
2016-10-28  000001.XSHE      54500.0
2016-10-31  000001.XSHE      54500.0
2016-11-01  000001.XSHE      54500.0
dtype: float64

In [22]:
# 每日账面已实现盈亏
perf.daily_pnl.head()

datetime
2016-10-26   -398.94
2016-10-27   -398.94
2016-10-28   -398.94
2016-10-31   -398.94
2016-11-01   -398.94
Name: daily_pnl, dtype: float64

In [25]:
# 每日账面浮动盈亏
perf.daily_float_pnl.head()

datetime
2016-10-26   -398.94
2016-10-27    146.06
2016-10-28    691.06
2016-10-31   -398.94
2016-11-01   -943.94
Name: daily_float_pnl, dtype: float64

In [26]:
# 每日账户总值
perf.daily_account_value.head()

datetime
2016-10-26     9999601.06
2016-10-27    10000146.06
2016-10-28    10000691.06
2016-10-31     9999601.06
2016-11-01     9999056.06
Name: daily_account_value, dtype: float64

In [27]:
# 每日账户净值
perf.daily_net.head()

datetime
2016-10-26    0.999960
2016-10-27    1.000015
2016-10-28    1.000069
2016-10-31    0.999960
2016-11-01    0.999906
Name: daily_net, dtype: float64

In [28]:
# 每日可用现金
perf.daily_cash.head()

datetime
2016-10-26    9500926.06
2016-10-27    9500926.06
2016-10-28    9500926.06
2016-10-31    9500926.06
2016-11-01    9500926.06
Name: daily_cash, dtype: float64

In [29]:
# 每日收益率
perf.daily_returns.head()

datetime
2016-10-26   -0.000040
2016-10-27    0.000055
2016-10-28    0.000054
2016-10-31   -0.000109
2016-11-01   -0.000055
Name: daily_returns, dtype: float64

In [30]:
# 每日各品种+现金的仓位占比
perf.positions_alloc.head()

,000001.XSHE,cash
datetime,,
2016-10-26,0.049869,0.950131
2016-10-27,0.049921,0.950079
2016-10-28,0.049973,0.950027
2016-10-31,0.049869,0.950131
2016-11-01,0.049818,0.950182


In [31]:
# 账户月回报
perf.monthly_returns(perf.daily_returns).head()

,1,2,3,4,5,10,11,12
2016,0.000,0.000,0.000,0.0,0.0,-0.0,0.002,-0.002
2017,0.001,0.001,-0.001,0.0,-0.0,0.0,0.000,0.000


In [32]:
# 账户年回报
perf.yearly_returns(perf.daily_returns).head()

,annual_returns
2016,-0.000312
2017,0.001013


In [33]:
# 账户累计收益
perf.cum_rets(perf.daily_returns).head()

datetime
2016-10-26    0.999960
2016-10-27    1.000015
2016-10-28    1.000069
2016-10-31    0.999960
2016-11-01    0.999906
Name: daily_returns, dtype: float64

In [39]:
# 每日持有股票品种数
perf.holdings(perf.daily_mv_df)["holdings by day"].head()

datetime
2016-10-26    1
2016-10-27    1
2016-10-28    1
2016-10-31    1
2016-11-01    1
dtype: int64

In [40]:
# 每月持有股票品种数
perf.holdings(perf.daily_mv_df)["holdings_by_month"].head()

datetime
2016-10-31    1.0
2016-11-30    1.0
2016-12-31    1.0
2017-01-31    1.0
2017-02-28    1.0
Freq: M, dtype: float64

In [41]:
# 平均持有股票品种数
perf.holdings(perf.daily_mv_df)["avg holdings"]

0.6736111111111112

In [42]:
# 前10最大回撤（通过top调节）
perf.drawdown_periods(perf.daily_returns, top=10)

,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,0.320689,2016-12-09,2016-12-28,NaT,NaN
1,0.0924119,2016-11-28,2016-12-05,2016-12-09,10
2,0.0544962,2016-10-28,2016-11-02,2016-11-11,11
3,0.0272392,2016-11-16,2016-11-18,2016-11-21,4
4,0,2016-10-26,2016-10-26,2016-10-26,1
5,0,2016-10-26,2016-10-26,2016-10-26,1
6,0,2016-10-26,2016-10-26,2016-10-26,1
7,0,2016-10-26,2016-10-26,2016-10-26,1
8,0,2016-10-26,2016-10-26,2016-10-26,1
9,0,2016-10-26,2016-10-26,2016-10-26,1


In [44]:
# 利润回吐比例
perf.drawdown_underwater(perf.daily_returns).head()

datetime
2016-10-26   -0.000000
2016-10-27   -0.000000
2016-10-28   -0.000000
2016-10-31   -0.000109
2016-11-01   -0.000163
Name: daily_returns, dtype: float64

In [45]:
# 绩效指标
hs300_rets = perf.benchmark_rets(code="000300.XSHG")
perf.perf_stats(perf.daily_returns, hs300_rets)

{'Backtest months': 6, 'perf_stats':                         Backtest
 Annual return               0.1%
 Cumulative returns          0.1%
 Annual volatility           0.4%
 Sharpe ratio            0.296852
 Calmar ratio            0.382293
 Stability              0.0237131
 Max drawdown               -0.3%
 Omega ratio              1.06672
 Sortino ratio           0.407502
 Skew                   -0.529328
 Kurtosis                  5.1098
 Tail ratio              0.978217
 Daily value at risk        -0.1%
 Alpha               -0.000578321
 Beta                    0.029071}

In [47]:
# 6个月、12个月滚动beta
perf.rolling_beta(perf.daily_returns,hs300_rets)

{'12-month': datetime
 2016-10-26   NaN
 2016-10-27   NaN
 2016-10-28   NaN
 2016-10-31   NaN
 2016-11-01   NaN
 2016-11-02   NaN
 2016-11-03   NaN
 2016-11-04   NaN
 2016-11-07   NaN
 2016-11-08   NaN
 2016-11-09   NaN
 2016-11-10   NaN
 2016-11-11   NaN
 2016-11-14   NaN
 2016-11-15   NaN
 2016-11-16   NaN
 2016-11-17   NaN
 2016-11-18   NaN
 2016-11-21   NaN
 2016-11-22   NaN
 2016-11-23   NaN
 2016-11-24   NaN
 2016-11-25   NaN
 2016-11-28   NaN
 2016-11-29   NaN
 2016-11-30   NaN
 2016-12-01   NaN
 2016-12-02   NaN
 2016-12-05   NaN
 2016-12-06   NaN
               ..
 2017-04-14   NaN
 2017-04-17   NaN
 2017-04-18   NaN
 2017-04-19   NaN
 2017-04-20   NaN
 2017-04-21   NaN
 2017-04-24   NaN
 2017-04-25   NaN
 2017-04-26   NaN
 2017-04-27   NaN
 2017-04-28   NaN
 2017-05-02   NaN
 2017-05-03   NaN
 2017-05-04   NaN
 2017-05-05   NaN
 2017-05-08   NaN
 2017-05-09   NaN
 2017-05-10   NaN
 2017-05-11   NaN
 2017-05-12   NaN
 2017-05-15   NaN
 2017-05-16   NaN
 2017-05-17   NaN
 2017-

In [48]:
# 六个月滚动波动率
perf.rolling_volatility(perf.daily_returns)

datetime
2016-10-26         NaN
2016-10-27         NaN
2016-10-28         NaN
2016-10-31         NaN
2016-11-01         NaN
2016-11-02         NaN
2016-11-03         NaN
2016-11-04         NaN
2016-11-07         NaN
2016-11-08         NaN
2016-11-09         NaN
2016-11-10         NaN
2016-11-11         NaN
2016-11-14         NaN
2016-11-15         NaN
2016-11-16         NaN
2016-11-17         NaN
2016-11-18         NaN
2016-11-21         NaN
2016-11-22         NaN
2016-11-23         NaN
2016-11-24         NaN
2016-11-25         NaN
2016-11-28         NaN
2016-11-29         NaN
2016-11-30         NaN
2016-12-01         NaN
2016-12-02         NaN
2016-12-05         NaN
2016-12-06         NaN
                ...   
2017-04-14         NaN
2017-04-17         NaN
2017-04-18         NaN
2017-04-19         NaN
2017-04-20         NaN
2017-04-21         NaN
2017-04-24         NaN
2017-04-25         NaN
2017-04-26         NaN
2017-04-27         NaN
2017-04-28         NaN
2017-05-02    0.004445
20

In [49]:
# 六个月滚动夏普比例
perf.rolling_sharpe(perf.daily_returns)

datetime
2016-10-26         NaN
2016-10-27         NaN
2016-10-28         NaN
2016-10-31         NaN
2016-11-01         NaN
2016-11-02         NaN
2016-11-03         NaN
2016-11-04         NaN
2016-11-07         NaN
2016-11-08         NaN
2016-11-09         NaN
2016-11-10         NaN
2016-11-11         NaN
2016-11-14         NaN
2016-11-15         NaN
2016-11-16         NaN
2016-11-17         NaN
2016-11-18         NaN
2016-11-21         NaN
2016-11-22         NaN
2016-11-23         NaN
2016-11-24         NaN
2016-11-25         NaN
2016-11-28         NaN
2016-11-29         NaN
2016-11-30         NaN
2016-12-01         NaN
2016-12-02         NaN
2016-12-05         NaN
2016-12-06         NaN
                ...   
2017-04-14         NaN
2017-04-17         NaN
2017-04-18         NaN
2017-04-19         NaN
2017-04-20         NaN
2017-04-21         NaN
2017-04-24         NaN
2017-04-25         NaN
2017-04-26         NaN
2017-04-27         NaN
2017-04-28         NaN
2017-05-02    0.335399
20

In [51]:
# 仓位比重(持仓市值/总市值)
perf.exposures(perf.daily_mv_df)

{'long_exp': datetime
 2016-10-26    0.049869
 2016-10-27    0.049921
 2016-10-28    0.049973
 2016-10-31    0.049869
 2016-11-01    0.049818
 2016-11-02    0.049455
 2016-11-03    0.049766
 2016-11-04    0.049662
 2016-11-07    0.049714
 2016-11-08    0.049869
 2016-11-09    0.049455
 2016-11-10    0.049818
 2016-11-11    0.050025
 2016-11-14    0.050232
 2016-11-15    0.050284
 2016-11-16    0.050284
 2016-11-17    0.050180
 2016-11-18    0.050025
 2016-11-21    0.050335
 2016-11-22    0.050956
 2016-11-23    0.051420
 2016-11-24    0.051524
 2016-11-25    0.052297
 2016-11-28    0.052349
 2016-11-29    0.052297
 2016-11-30    0.051936
 2016-12-01    0.052194
 2016-12-02    0.051936
 2016-12-05    0.051472
 2016-12-06    0.051627
                 ...   
 2017-04-14    0.000000
 2017-04-17    0.000000
 2017-04-18    0.000000
 2017-04-19    0.000000
 2017-04-20    0.000000
 2017-04-21    0.000000
 2017-04-24    0.000000
 2017-04-25    0.000000
 2017-04-26    0.000000
 2017-04-27    0.0

In [53]:
# 历史重/持仓股(某一日达到该日top要求即入选)
perf.top_positions(perf.daily_mv_df,
                   top=10)

{'all': 000001.XSHE    0.052452
 dtype: float64, 'top_abs': 000001.XSHE    0.052452
 dtype: float64, 'top_long': 000001.XSHE    0.052452
 dtype: float64, 'top_short': Series([], dtype: float64)}

In [55]:
# 每天持有个股的最大头寸占比和头寸占比众数（分多空头)
perf.max_median_position_concentration(perf.daily_mv_df).head()

,max_long,median_long,median_short,max_short
datetime,,,,
2016-10-26,0.049869,0.049869,NaN,NaN
2016-10-27,0.049921,0.049921,NaN,NaN
2016-10-28,0.049973,0.049973,NaN,NaN
2016-10-31,0.049869,0.049869,NaN,NaN
2016-11-01,0.049818,0.049818,NaN,NaN


In [57]:
# 行业风险暴露（行业资金分配）
symbol_sector_map = perf.symbol_sector_map()
perf.sector_exposures(perf.daily_mv_df, symbol_sector_map).head()

,金融行业,cash
datetime,,
2016-10-26,498675.0,9500926.06
2016-10-27,499220.0,9500926.06
2016-10-28,499765.0,9500926.06
2016-10-31,498675.0,9500926.06
2016-11-01,498130.0,9500926.06


In [58]:
# 按样本内外的日度、月度、周度来呈现收益分布特征
perf.return_quantiles(perf.daily_returns,live_start_date=None)

{'is': {'Daily': datetime
  2016-10-26   -0.000040
  2016-10-27    0.000055
  2016-10-28    0.000054
  2016-10-31   -0.000109
  2016-11-01   -0.000055
  2016-11-02   -0.000382
  2016-11-03    0.000327
  2016-11-04   -0.000109
  2016-11-07    0.000055
  2016-11-08    0.000164
  2016-11-09   -0.000436
  2016-11-10    0.000382
  2016-11-11    0.000218
  2016-11-14    0.000218
  2016-11-15    0.000054
  2016-11-16    0.000000
  2016-11-17   -0.000109
  2016-11-18   -0.000163
  2016-11-21    0.000327
  2016-11-22    0.000654
  2016-11-23    0.000490
  2016-11-24    0.000109
  2016-11-25    0.000816
  2016-11-28    0.000054
  2016-11-29   -0.000054
  2016-11-30   -0.000381
  2016-12-01    0.000272
  2016-12-02   -0.000272
  2016-12-05   -0.000489
  2016-12-06    0.000163
                  ...   
  2017-04-14    0.000000
  2017-04-17    0.000000
  2017-04-18    0.000000
  2017-04-19    0.000000
  2017-04-20    0.000000
  2017-04-21    0.000000
  2017-04-24    0.000000
  2017-04-25    0.000000

In [60]:
# 换手率
perf.turnover(perf.transactions,perf.daily_mv_df)

{'Daily': datetime
 2016-10-26    0.024935
 2016-10-27    0.000000
 2016-10-28    0.000000
 2016-10-31    0.000000
 2016-11-01    0.000000
 2016-11-02    0.000000
 2016-11-03    0.000000
 2016-11-04    0.000000
 2016-11-07    0.000000
 2016-11-08    0.000000
 2016-11-09    0.000000
 2016-11-10    0.000000
 2016-11-11    0.000000
 2016-11-14    0.000000
 2016-11-15    0.000000
 2016-11-16    0.000000
 2016-11-17    0.000000
 2016-11-18    0.000000
 2016-11-21    0.000000
 2016-11-22    0.000000
 2016-11-23    0.000000
 2016-11-24    0.000000
 2016-11-25    0.000000
 2016-11-28    0.000000
 2016-11-29    0.000000
 2016-11-30    0.000000
 2016-12-01    0.000000
 2016-12-02    0.000000
 2016-12-05    0.000000
 2016-12-06    0.000000
                 ...   
 2017-04-14    0.000000
 2017-04-17    0.000000
 2017-04-18    0.000000
 2017-04-19    0.000000
 2017-04-20    0.000000
 2017-04-21    0.000000
 2017-04-24    0.000000
 2017-04-25    0.000000
 2017-04-26    0.000000
 2017-04-27    0.0000

In [62]:
# 不同滑点下的净值曲线
# 注：1bps = 0.0001
# A股一个点是0.01 即100bps
perf.slippage_sweep(perf.daily_returns,
                    perf.transactions,
                    perf.daily_mv_df,
                    slippage_params=(100, 200, 300, 400, 500)).head()

,100 bps,200 bps,300 bps,400 bps,500 bps
datetime,,,,,
2016-10-26,0.999461,0.998963,0.998464,0.997965,0.997467
2016-10-27,0.999516,0.999017,0.998518,0.998020,0.997521
2016-10-28,0.999570,0.999072,0.998573,0.998074,0.997575
2016-10-31,0.999461,0.998963,0.998464,0.997965,0.997467
2016-11-01,0.999407,0.998908,0.998410,0.997911,0.997412


In [64]:
# 滑点敏感度 —— 不同滑点下对应的年化收益率
perf.slippage_sensitivity(perf.daily_returns,
                          perf.transactions,
                          perf.daily_mv_df,
                          slippage_range = range(100,5000,100)).head()

100   -0.001419
200   -0.004058
300   -0.006691
400   -0.009319
500   -0.011942
dtype: float64

In [76]:
# 考虑资金量带来市场冲击后的修正夏普率
# 默认入场资金从100000 - 300000000， 步长1000000
# 返回指定资金量下的修正夏普率
market_data = perf.market_data_panel().rename({"close":"price"})
perf.capacity_sweep(perf.daily_returns,
                    perf.transactions,
                    market_data,
                    bt_starting_capital=10000000,
                    min_pv=100000,
                    max_pv=300000000,
                    step_size=1000000)

0.1      0.296852
1.1      0.296852
2.1      0.296852
3.1      0.296852
4.1      0.296852
5.1      0.296851
6.1      0.296851
7.1      0.296850
8.1      0.296850
9.1      0.296849
10.1     0.296848
11.1     0.296847
12.1     0.296846
13.1     0.296845
14.1     0.296844
15.1     0.296843
16.1     0.296841
17.1     0.296840
18.1     0.296838
19.1     0.296837
20.1     0.296835
21.1     0.296833
22.1     0.296831
23.1     0.296829
24.1     0.296827
25.1     0.296825
26.1     0.296823
27.1     0.296821
28.1     0.296818
29.1     0.296816
           ...   
270.1    0.293701
271.1    0.293677
272.1    0.293654
273.1    0.293630
274.1    0.293607
275.1    0.293583
276.1    0.293559
277.1    0.293535
278.1    0.293511
279.1    0.293487
280.1    0.293463
281.1    0.293439
282.1    0.293415
283.1    0.293390
284.1    0.293366
285.1    0.293341
286.1    0.293316
287.1    0.293292
288.1    0.293267
289.1    0.293242
290.1    0.293217
291.1    0.293192
292.1    0.293166
293.1    0.293141
294.1    0